In [1]:
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Amroy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Amroy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Amroy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
df_train = pd.read_json(r"D:\git\Taxonomy_LLM_models\icecat_data_train.json")
df_train.head()

,Brand,BrandInfo.BrandLocalName,BrandInfo.BrandLogo,BrandInfo.BrandName,BrandLogo,BrandPartCode,BulletPoints,Category.CategoryID,Category.Name.Language,Category.Name.Value,...,ProductSeries.Language,ProductSeries.Value,BulletPoints.BulletPointsId,BulletPoints.Language,BulletPoints.Updated,BulletPoints.Values,VirtualCategory,SummaryDescription,pathlist_ids,pathlist_names
1072689,ASUS,,https://images.icecat.biz/img/brand/thumb/161_...,ASUS,https://images.icecat.biz/img/brand/thumb/161_...,K31CD-IT049T,[],153,EN,PCs/Workstations,...,None,None,NaN,None,None,None,"[{'VirtualCategoryID': 195, 'UNCATID': '431718...",NaN,2833>150>153,Computers & Electronics>Computers>PCs/Workstat...
906402,HP,,https://images.icecat.biz/img/brand/thumb/1_91...,HP,https://images.icecat.biz/img/brand/thumb/1_91...,686915-A41,[],2509,EN,Notebook Spare Parts,...,None,None,NaN,None,None,None,None,NaN,2833>150>8355>2509,Computers & Electronics>Computers>Notebook Par...
411281,C2G,,https://images.icecat.biz/img/brand/thumb/2834...,C2G,https://images.icecat.biz/img/brand/thumb/2834...,37745,[],953,EN,Fibre Optic Cables,...,None,None,NaN,None,None,None,None,NaN,2833>830>953,Computers & Electronics>Computer Cables>Fibre ...
425903,HP,,https://images.icecat.biz/img/brand/thumb/1_91...,HP,https://images.icecat.biz/img/brand/thumb/1_91...,FA889AA#AC3,[],8194,EN,Handheld Mobile Computer Spare Parts,...,None,None,NaN,None,None,None,None,NaN,2833>150>8194,Computers & Electronics>Computers>Handheld Mob...
1047582,Lenovo,,https://images.icecat.biz/img/brand/thumb/728_...,Lenovo,https://images.icecat.biz/img/brand/thumb/728_...,109559U,[],153,EN,PCs/Workstations,...,None,None,NaN,None,None,None,"[{'VirtualCategoryID': 194, 'UNCATID': '431718...",NaN,2833>150>153,Computers & Electronics>Computers>PCs/Workstat...


In [ ]:
df_validate = pd.read_json(r"D:\git\Taxonomy_LLM_models\icecat_data_validate.json")
df_validate.head()

,Brand,BrandInfo.BrandLocalName,BrandInfo.BrandLogo,BrandInfo.BrandName,BrandLogo,BrandPartCode,BulletPoints,Category.CategoryID,Category.Name.Language,Category.Name.Value,...,ProductSeries.Language,ProductSeries.Value,BulletPoints.BulletPointsId,BulletPoints.Language,BulletPoints.Updated,BulletPoints.Values,VirtualCategory,SummaryDescription,pathlist_ids,pathlist_names
309544,Fujitsu,,https://images.icecat.biz/img/brand/thumb/15_b...,Fujitsu,https://images.icecat.biz/img/brand/thumb/15_b...,S26361-F5524-L800,[],1563,EN,Internal Solid State Drives,...,None,None,NaN,None,None,None,None,NaN,2833>206>2840>1563,Computers & Electronics>Data Storage>Data Stor...
805366,PanzerGlass,,https://images.icecat.biz/img/brand/thumb/1016...,PanzerGlass,https://images.icecat.biz/img/brand/thumb/1016...,PG1501,[],1568,EN,Screen Protectors,...,None,None,NaN,None,None,None,None,NaN,2833>107>1568,Computers & Electronics>Telecom & Navigation>S...
126809,2-Power,,https://images.icecat.biz/img/brand/thumb/1520...,2-Power,https://images.icecat.biz/img/brand/thumb/1520...,ALT268563B,[],911,EN,Memory Modules,...,None,None,NaN,None,None,None,None,NaN,2833>106>2844>911,Computers & Electronics>Computer Components>Sy...
922232,Lenovo,,https://images.icecat.biz/img/brand/thumb/728_...,Lenovo,https://images.icecat.biz/img/brand/thumb/728_...,25205062,[],2509,EN,Notebook Spare Parts,...,None,None,NaN,None,None,None,None,NaN,2833>150>8355>2509,Computers & Electronics>Computers>Notebook Par...
567207,Verbatim,,https://images.icecat.biz/img/brand/thumb/669_...,Verbatim,https://images.icecat.biz/img/brand/thumb/669_...,97537,[],194,EN,Keyboards,...,None,None,NaN,None,None,None,None,NaN,2833>191>194,Computers & Electronics>Data Input Devices>Key...


In [6]:
df_test = pd.read_json(r"D:\git\Taxonomy_LLM_models\icecat_data_test.json")
df_test.head()

,Brand,BrandInfo.BrandLocalName,BrandInfo.BrandLogo,BrandInfo.BrandName,BrandLogo,BrandPartCode,BulletPoints,Category.CategoryID,Category.Name.Language,Category.Name.Value,...,ProductSeries.Language,ProductSeries.Value,BulletPoints.BulletPointsId,BulletPoints.Language,BulletPoints.Updated,BulletPoints.Values,VirtualCategory,SummaryDescription,pathlist_ids,pathlist_names
500081,Fujitsu,,https://images.icecat.biz/img/brand/thumb/15_b...,Fujitsu,https://images.icecat.biz/img/brand/thumb/15_b...,FSP:G-SW3Z560PRE0S,[],788,EN,Warranty & Support Extensions,...,None,None,NaN,None,None,None,None,NaN,2833>839>788,Computers & Electronics>Warranty & Support>War...
741063,HP,,https://images.icecat.biz/img/brand/thumb/1_91...,HP,https://images.icecat.biz/img/brand/thumb/1_91...,5KM83PA,None,151,EN,Notebooks,...,EN,13,1268560.0,EN,2018-10-21 21:55:51,"[Windows 10 Home 64-bit, Intel® Core™ i7-8565U...","[{'VirtualCategoryID': 329, 'UNCATID': '432115...",NaN,2833>150>151,Computers & Electronics>Computers>Notebooks
1091454,HP,,https://images.icecat.biz/img/brand/thumb/1_91...,HP,https://images.icecat.biz/img/brand/thumb/1_91...,4NG33EA,[],153,EN,PCs/Workstations,...,EN,880,NaN,None,None,None,"[{'VirtualCategoryID': 195, 'UNCATID': '431718...",NaN,2833>150>153,Computers & Electronics>Computers>PCs/Workstat...
522928,C2G,,https://images.icecat.biz/img/brand/thumb/2834...,C2G,https://images.icecat.biz/img/brand/thumb/2834...,83061,[],883,EN,Networking Cables,...,None,None,NaN,None,None,None,None,NaN,2833>830>883,Computers & Electronics>Computer Cables>Networ...
479478,Lenovo,,https://images.icecat.biz/img/brand/thumb/728_...,Lenovo,https://images.icecat.biz/img/brand/thumb/728_...,5PS0A14091,[],788,EN,Warranty & Support Extensions,...,None,None,NaN,None,None,None,None,NaN,2833>839>788,Computers & Electronics>Warranty & Support>War...


In [7]:
df_train.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 489902 entries, 1072689 to 1008771
Data columns (total 45 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Brand                                       489902 non-null  object 
 1   BrandInfo.BrandLocalName                    489902 non-null  object 
 2   BrandInfo.BrandLogo                         488765 non-null  object 
 3   BrandInfo.BrandName                         489902 non-null  object 
 4   BrandLogo                                   488765 non-null  object 
 5   BrandPartCode                               489902 non-null  object 
 6   BulletPoints                                455119 non-null  object 
 7   Category.CategoryID                         489902 non-null  int64  
 8   Category.Name.Language                      489902 non-null  object 
 9   Category.Name.Value                         489902 non-null  ob

In [8]:
df_train.isna().sum() #check for null values

Brand                                              0
BrandInfo.BrandLocalName                           0
BrandInfo.BrandLogo                             1137
BrandInfo.BrandName                                0
BrandLogo                                       1137
BrandPartCode                                      0
BulletPoints                                   34783
Category.CategoryID                                0
Category.Name.Language                             0
Category.Name.Value                                0
Description.Disclaimer                         53728
Description.ID                                 53728
Description.LeafletPDFURL                      53728
Description.LongDesc                           53728
Description.LongProductName                    53728
Description.ManualPDFSize                      61104
Description.ManualPDFURL                       53728
Description.MiddleDesc                         53728
Description.PDFSize                           

In [9]:
# Show all columns
pd.set_option('display.max_columns', None)
print(df_train.head()) 

          Brand BrandInfo.BrandLocalName  \
1072689    ASUS                            
906402       HP                            
411281      C2G                            
425903       HP                            
1047582  Lenovo                            

                                       BrandInfo.BrandLogo  \
1072689  https://images.icecat.biz/img/brand/thumb/161_...   
906402   https://images.icecat.biz/img/brand/thumb/1_91...   
411281   https://images.icecat.biz/img/brand/thumb/2834...   
425903   https://images.icecat.biz/img/brand/thumb/1_91...   
1047582  https://images.icecat.biz/img/brand/thumb/728_...   

        BrandInfo.BrandName  \
1072689                ASUS   
906402                   HP   
411281                  C2G   
425903                   HP   
1047582              Lenovo   

                                                 BrandLogo BrandPartCode  \
1072689  https://images.icecat.biz/img/brand/thumb/161_...  K31CD-IT049T   
906402   https://images

In [10]:
df_train=df_train[["Title","Description.LongDesc","SummaryDescription.LongSummaryDescription","SummaryDescription.ShortSummaryDescription","Description.LongProductName","pathlist_names"]]
df_train

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names
1072689,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,<b>Smart Multimedia Performance</b><br>\nVivoP...,ASUS K31CD-IT049T. Processor frequency: 3.4 GH...,"ASUS K31CD-IT049T, 3.4 GHz, 6th gen Intel® Cor...","Intel Core i7-6700 (8M Cache, 3.4GHz), 16GB RA...",Computers & Electronics>Computers>PCs/Workstat...
906402,HP 686915-A41 notebook spare part Keyboard,,HP 686915-A41. Type: Keyboard. Keyboard langua...,"HP 686915-A41, Keyboard, Belgian, Keyboard bac...",Keyboard in midnight black finish with backlig...,Computers & Electronics>Computers>Notebook Par...
411281,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,Get the performance you demand at a price that...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,1m ST/SC Plenum-Rated 9/125 Duplex Single-Mode...,Computers & Electronics>Computer Cables>Fibre ...
425903,HP FA889AA Battery,Keeping an extra source of power nearby means ...,"HP FA889AA. Product type: Battery, Product col...","HP FA889AA, Battery, White, Lithium-Ion (Li-Io...","1100 mAh, Lithium Ion, Standard Battery",Computers & Electronics>Computers>Handheld Mob...
1047582,Lenovo ThinkStation C30 Intel® Xeon® E5 Family...,The C30 builds on its award-winning design as ...,Lenovo ThinkStation C30. Processor frequency: ...,"Lenovo ThinkStation C30, 2 GHz, Intel® Xeon® E...","Intel Xeon E5-2620 (15M Cache, 2.00 GHz, 7.20 ...",Computers & Electronics>Computers>PCs/Workstat...
...,...,...,...,...,...,...
1093316,Fujitsu ESPRIMO P758 8th gen Intel® Core™ i7 i...,The FUJITSU ESPRIMO P758 Desktop provides high...,Fujitsu ESPRIMO P758. Processor frequency: 3.2...,"Fujitsu ESPRIMO P758, 3.2 GHz, 8th gen Intel® ...","Intel Core i7-8700, 1 x 16 GB DDR4 2666 MHz, P...",Computers & Electronics>Computers>PCs/Workstat...
886679,Acer 55.PDS07.001 notebook spare part Power board,,"Acer 55.PDS07.001. Type: Power board, Brand co...","Acer 55.PDS07.001, Power board, Acer",Power board,Computers & Electronics>Computers>Notebook Par...
35003,Gigabyte Aire M1 mouse USB Optical 1000 DPI,<b>Mini Size</b><br>\nUltimate Choice for Note...,Gigabyte Aire M1. Movement detection technolog...,"Gigabyte Aire M1, Optical, USB, 1000 DPI, Black","Optical, USB, 1000dpi, 61g, Black",Computers & Electronics>Data Input Devices>Mice
779133,HP Pavilion x360 14-dh0062no Gold Hybrid (2-in...,None,HP Pavilion x360 14-dh0062no. Product type: Hy...,"HP Pavilion x360 14-dh0062no, 8th gen Intel® C...",None,Computers & Electronics>Computers>Notebooks


In [11]:
df_validate=df_validate[["Title","Description.LongDesc","SummaryDescription.LongSummaryDescription","SummaryDescription.ShortSummaryDescription","Description.LongProductName","pathlist_names"]]
df_validate

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names
309544,Fujitsu S26361-F5524-L800 internal solid state...,Fujitsu offers a full line of Hard Drives and ...,Fujitsu S26361-F5524-L800. SSD capacity: 800 G...,"Fujitsu S26361-F5524-L800, 800 GB, 2.5"", 6 Gbit/s","800GB Read-Inten 6.35 cm (2.5"") , SATA III, Ho...",Computers & Electronics>Data Storage>Data Stor...
805366,PanzerGlass PG1501 screen protector Clear scre...,,PanzerGlass PG1501. Type: Clear screen protect...,"PanzerGlass PG1501, Clear screen protector, Mo...",Protective Anti Scratch Fluid Resistant Glass ...,Computers & Electronics>Telecom & Navigation>S...
126809,2-Power ALT268563B memory module 8 GB DDR4 266...,<b>2-POWER MEMORY</b><br>\n2-Power memory modu...,"2-Power ALT268563B. Component for: PC/server, ...","2-Power ALT268563B, 8 GB, 1 x 8 GB, DDR4, 2666...",8GB DDR4 2666MHz Replaces 2P-L09284-850,Computers & Electronics>Computer Components>Sy...
922232,Lenovo 25205062 notebook spare part Keyboard,- High quality<br>\n- Long life<br>\n- Easy in...,Lenovo 25205062. Type: Keyboard. Keyboard lang...,"Lenovo 25205062, Keyboard, Dutch, Lenovo, Idea...",Keyboard for Ideapad S300,Computers & Electronics>Computers>Notebook Par...
567207,Verbatim 97537 keyboard Bluetooth QWERTY Black,"On the go, looking for a quick and easy way to...",Verbatim 97537. Connectivity technology: Wirel...,"Verbatim 97537, Wireless, Bluetooth, QWERTY, B...","Wireless, Bluetooth, iOS 4, f/iPad/iPhone/iPod...",Computers & Electronics>Data Input Devices>Key...
...,...,...,...,...,...,...
488675,HP 1y PW Chnl Rmt Parts LJ M604 Support,Parts and Remote Service offers access to high...,HP 1y PW Chnl Rmt Parts LJ M604 Support,HP 1y PW Chnl Rmt Parts LJ M604 Support,1y PW Chnl Rmt Parts LJ M604 Support,Computers & Electronics>Warranty & Support>War...
735977,HP EliteBook x360 1030 G3 + Zero-X Raven+ Dron...,The number of devices you need to make an impa...,HP EliteBook x360 1030 G3 + Zero-X Raven+ Dron...,HP EliteBook x360 1030 G3 + Zero-X Raven+ Dron...,"Intel Core i7-8550U (1.8GHz, 8MB), 33.782 cm (...",Computers & Electronics>Computers>Notebooks
643852,"HP Chromebook 14 G3 Black 35.6 cm (14"") 1366 x...","Connect in style. Open your sleek, next-genera...","HP Chromebook 14 G3. Product type: Chromebook,...","HP Chromebook 14 G3, NVIDIA Tegra, 2.3 GHz, 35...","NVIDIA Tegra K1 (2.3GHz), 35.56 cm (14"") HD (1...",Computers & Electronics>Computers>Notebooks
656159,ASUS ROG G551JX-DM218H notebook Black 39.6 cm ...,<b>High-performance gaming laptop with enthusi...,ASUS ROG G551JX-DM218H. Product type: Notebook...,"ASUS ROG G551JX-DM218H, 4th gen Intel® Core™ i...","Intel Core i7-4710HQ (2.5GHz, 6M Cache), 8GB D...",Computers & Electronics>Computers>Notebooks


In [12]:
df_test=df_test[["Title","Description.LongDesc","SummaryDescription.LongSummaryDescription","SummaryDescription.ShortSummaryDescription","Description.LongProductName","pathlist_names"]]
df_test

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names
500081,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...,In times of growing complexity and decreasing ...,Fujitsu FSP:G-SW3Z560PRE0S. Number of years: 3...,"Fujitsu FSP:G-SW3Z560PRE0S, 3 year(s), 9x5","SP 3y TS Sub & Upgr, 9x5, 4h Rm Rt f/ CS200c A...",Computers & Electronics>Warranty & Support>War...
741063,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid...",<b>Revolutionary battery life on a convertible...,HP Spectre x360 13-ap0023tu. Product type: Hyb...,"HP Spectre x360 13-ap0023tu, 8th gen Intel® Co...","Intel® Core™ i7-8565U (1.8 GHz), 16GB DDR4-SDR...",Computers & Electronics>Computers>Notebooks
1091454,HP OMEN 880-156nf 8th gen Intel® Core™ i7 i7-8...,None,HP OMEN 880-156nf. Processor frequency: 3.2 GH...,"HP OMEN 880-156nf, 3.2 GHz, 8th gen Intel® Cor...",None,Computers & Electronics>Computers>PCs/Workstat...
522928,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,Perfect for your home office or a large instal...,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,Cat5E Assembled UTP Patch Cable Green 1m,Computers & Electronics>Computer Cables>Networ...
479478,Lenovo 5PS0A14091 warranty/support extension,Lenovo offers a comprehensive portfolio of val...,"Lenovo 5PS0A14091. Number of years: 3 year(s),...","Lenovo 5PS0A14091, 3 year(s), On-site, 24x7, N...",3YR Onsite + Keep Your Drive,Computers & Electronics>Warranty & Support>War...
...,...,...,...,...,...,...
931939,Lenovo 90006646 notebook spare part Motherboard,- High quality<br>\n- Long life,"Lenovo 90006646. Type: Motherboard, Brand comp...","Lenovo 90006646, Motherboard, Lenovo, S410p",Motherboard for Lenovo S410p notebook,Computers & Electronics>Computers>Notebook Par...
456002,i-tec LAN2RS232 interface cards/adapter,Adapter Ethernet to serial RS-232 (DB-9) allow...,"i-tec LAN2RS232. Host interface: Serial, Seria...","i-tec LAN2RS232, Serial, RS-232, Black,White, ...",LAN Server - Serial RS232,Computers & Electronics>Computer Components>Sy...
897175,HP 714519-001 notebook spare part Motherboard,System board equipped with an Intel Dual Core ...,"HP 714519-001. Type: Motherboard, Brand compat...","HP 714519-001, Motherboard, HP, EliteBook 2170p",System board (includes replacement thermal mat...,Computers & Electronics>Computers>Notebook Par...
517954,HP HF9Y6E warranty/support extension,HPE Foundation Care Service is composed of com...,HP HF9Y6E,HP HF9Y6E,"Web Knowledge Base Access, Repair, Remote Diag...",Computers & Electronics>Warranty & Support>War...


In [13]:
# Keep rows with exactly 2 '>' symbols in the path (i.e., 3 nodes)
df_train = df_train[df_train['pathlist_names'].str.count('>') == 2].reset_index(drop=True)
df_validate = df_validate[df_validate['pathlist_names'].str.count('>') == 2].reset_index(drop=True)
df_test = df_test[df_test['pathlist_names'].str.count('>') == 2].reset_index(drop=True)



In [14]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289865 entries, 0 to 289864
Data columns (total 6 columns):
 #   Column                                      Non-Null Count   Dtype 
---  ------                                      --------------   ----- 
 0   Title                                       289865 non-null  object
 1   Description.LongDesc                        244918 non-null  object
 2   SummaryDescription.LongSummaryDescription   289865 non-null  object
 3   SummaryDescription.ShortSummaryDescription  289865 non-null  object
 4   Description.LongProductName                 244918 non-null  object
 5   pathlist_names                              289865 non-null  object
dtypes: object(6)
memory usage: 13.3+ MB


In [15]:
df_validate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72513 entries, 0 to 72512
Data columns (total 6 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Title                                       72513 non-null  object
 1   Description.LongDesc                        61240 non-null  object
 2   SummaryDescription.LongSummaryDescription   72513 non-null  object
 3   SummaryDescription.ShortSummaryDescription  72513 non-null  object
 4   Description.LongProductName                 61240 non-null  object
 5   pathlist_names                              72513 non-null  object
dtypes: object(6)
memory usage: 3.3+ MB


In [16]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90598 entries, 0 to 90597
Data columns (total 6 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Title                                       90598 non-null  object
 1   Description.LongDesc                        76603 non-null  object
 2   SummaryDescription.LongSummaryDescription   90598 non-null  object
 3   SummaryDescription.ShortSummaryDescription  90598 non-null  object
 4   Description.LongProductName                 76603 non-null  object
 5   pathlist_names                              90598 non-null  object
dtypes: object(6)
memory usage: 4.1+ MB


In [17]:
print(df_train.isna().sum())
print(df_validate.isna().sum())
print(df_test.isna().sum())

Title                                             0
Description.LongDesc                          44947
SummaryDescription.LongSummaryDescription         0
SummaryDescription.ShortSummaryDescription        0
Description.LongProductName                   44947
pathlist_names                                    0
dtype: int64
Title                                             0
Description.LongDesc                          11273
SummaryDescription.LongSummaryDescription         0
SummaryDescription.ShortSummaryDescription        0
Description.LongProductName                   11273
pathlist_names                                    0
dtype: int64
Title                                             0
Description.LongDesc                          13995
SummaryDescription.LongSummaryDescription         0
SummaryDescription.ShortSummaryDescription        0
Description.LongProductName                   13995
pathlist_names                                    0
dtype: int64


In [18]:
# Fill NaN with empty strings
df_train.fillna("", inplace=True)

In [19]:
df_train.isna().sum()

Title                                         0
Description.LongDesc                          0
SummaryDescription.LongSummaryDescription     0
SummaryDescription.ShortSummaryDescription    0
Description.LongProductName                   0
pathlist_names                                0
dtype: int64

In [ ]:
# Fill NaN with empty strings
df_validate.fillna("", inplace=True)

In [20]:
# Fill NaN with empty strings
df_test.fillna("", inplace=True)

In [22]:
# Combine 5 textual columns into one
def combine_text(row):
    return " ".join([
        row["Title"],
        row["Description.LongDesc"],
        row["SummaryDescription.LongSummaryDescription"],
        row["SummaryDescription.ShortSummaryDescription"],
        row["Description.LongProductName"]
        #row["pathlist_names"]
    ])

In [23]:
df_train['raw_text'] = df_train.apply(combine_text,axis=1)

In [24]:
df_validate['raw_text'] = df_validate.apply(combine_text,axis=1)

In [25]:
df_test['raw_text'] = df_test.apply(combine_text,axis=1)

In [26]:
df_train.head(2)

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text
0,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,<b>Smart Multimedia Performance</b><br>\nVivoP...,ASUS K31CD-IT049T. Processor frequency: 3.4 GH...,"ASUS K31CD-IT049T, 3.4 GHz, 6th gen Intel® Cor...","Intel Core i7-6700 (8M Cache, 3.4GHz), 16GB RA...",Computers & Electronics>Computers>PCs/Workstat...,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...
1,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,Get the performance you demand at a price that...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,1m ST/SC Plenum-Rated 9/125 Duplex Single-Mode...,Computers & Electronics>Computer Cables>Fibre ...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...


In [27]:
# Initialize NLTK tools
stop = set(stopwords.words("english"))
lemma = WordNetLemmatizer()
punc = set(punctuation)

In [28]:
# Cleaning function
def clean_text(text):
    tokens = word_tokenize(text.lower())
    word_tokens = [t for t in tokens if t.isalpha()]
    clean_tokens = [t for t in word_tokens if t not in stop]
    lemmatized_tokens = [lemma.lemmatize(t, pos="n") for t in clean_tokens]
    return " ".join(lemmatized_tokens)

In [29]:
from tqdm import tqdm
tqdm.pandas()
df_train["cleaned_text"] = df_train["raw_text"].progress_apply(clean_text)
df_validate["cleaned_text"] = df_validate["raw_text"].progress_apply(clean_text)
df_test["cleaned_text"] = df_test["raw_text"].progress_apply(clean_text)

100%|██████████| 90598/90598 [02:44<00:00, 551.97it/s]


In [30]:
df_train[['raw_text','cleaned_text']]

,raw_text,cleaned_text
0,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,asus pc gen gb gb hdd black tower b smart mult...
1,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,duplex fiber patch cable blue fiber optic cabl...
2,HP FA889AA Battery Keeping an extra source of ...,hp battery keeping extra source power nearby m...
3,Lenovo ThinkStation C30 Intel® Xeon® E5 Family...,lenovo thinkstation family gb gb hdd black tow...
4,Lenovo 00D8199 software license/upgrade SUSE L...,lenovo software suse linux enterprise server s...
...,...,...
289860,Duracell 16 LR03 AAA Single-use battery Alkali...,duracell aaa battery alkaline duracell aaa bat...
289861,C2G 2m Velocity Camcorder Audio/Video Cable 3 ...,velocity camcorder cable x rca black camcorder...
289862,Fujitsu ESPRIMO P758 8th gen Intel® Core™ i7 i...,fujitsu esprimo gen gb gb ssd black red midi t...
289863,Gigabyte Aire M1 mouse USB Optical 1000 DPI <b...,gigabyte aire mouse usb optical dpi b mini siz...


In [31]:
df_validate[['raw_text','cleaned_text']]

,raw_text,cleaned_text
0,PanzerGlass PG1501 screen protector Clear scre...,panzerglass screen protector clear screen prot...
1,Verbatim 97537 keyboard Bluetooth QWERTY Black...,verbatim keyboard bluetooth qwerty black go lo...
2,"Symantec Backup Exec 2012 Agent f/ Windows, 1u...",symantec backup exec agent window initial e rn...
3,"APC Change Mgr, 1 Year Software Maintenance Co...",apc change mgr year software maintenance contr...
4,Urban Factory Adapter mini display port to HDM...,urban factory adapter mini display port hdmi m...
...,...,...
72508,Acer Aspire E E5-774G-57P5 Black Notebook 43.9...,acer aspire e black notebook cm x pixel gen gb...
72509,HP 1y PW Chnl Rmt Parts LJ M604 Support Parts ...,hp pw chnl rmt part lj support part remote ser...
72510,HP EliteBook x360 1030 G3 + Zero-X Raven+ Dron...,hp elitebook drone silver hybrid cm x pixel to...
72511,"HP Chromebook 14 G3 Black 35.6 cm (14"") 1366 x...",hp chromebook black cm x pixel nvidia tegra gb...


In [32]:
df_test[['raw_text','cleaned_text']]

,raw_text,cleaned_text
0,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...,fujitsu fsp extension time growing complexity ...
1,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid...",hp spectre blue silver hybrid cm x pixel touch...
2,HP OMEN 880-156nf 8th gen Intel® Core™ i7 i7-8...,hp omen gen gb gb black tower pc hp omen proce...
3,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,unshielded utp network patch cable green perfe...
4,Lenovo 5PS0A14091 warranty/support extension L...,lenovo extension lenovo offer comprehensive po...
...,...,...
90593,HP H8B37A4#YZJ warranty/support extension An e...,hp yzj extension enhanced call experience incl...
90594,HP 784415-211 mobile device keyboard Hungarian...,hp mobile device keyboard hungarian black grey...
90595,Acer Liquid Z3 FlipCase mobile phone case Whit...,acer liquid flipcase mobile phone case white b...
90596,HP HF9Y6E warranty/support extension HPE Found...,hp extension hpe foundation care service compo...


In [ ]:
# df_train.to_csv("icecat_preprocessed_train.csv", index=False)

In [ ]:
# df_train.to_json("icecat_preprocessed_train.json")

In [ ]:
# df_train.to_json("icecat_preprocessed_train_3_Categories.json")
# df_validate.to_json("icecat_preprocessed_validate_3_Categories.json")
# df_test.to_json("icecat_preprocessed_test_3_Categories.json")

In [36]:
df_test.head(3)

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text,cleaned_text
0,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...,In times of growing complexity and decreasing ...,Fujitsu FSP:G-SW3Z560PRE0S. Number of years: 3...,"Fujitsu FSP:G-SW3Z560PRE0S, 3 year(s), 9x5","SP 3y TS Sub & Upgr, 9x5, 4h Rm Rt f/ CS200c A...",Computers & Electronics>Warranty & Support>War...,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...,fujitsu fsp extension time growing complexity ...
1,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid...",<b>Revolutionary battery life on a convertible...,HP Spectre x360 13-ap0023tu. Product type: Hyb...,"HP Spectre x360 13-ap0023tu, 8th gen Intel® Co...","Intel® Core™ i7-8565U (1.8 GHz), 16GB DDR4-SDR...",Computers & Electronics>Computers>Notebooks,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid...",hp spectre blue silver hybrid cm x pixel touch...
2,HP OMEN 880-156nf 8th gen Intel® Core™ i7 i7-8...,,HP OMEN 880-156nf. Processor frequency: 3.2 GH...,"HP OMEN 880-156nf, 3.2 GHz, 8th gen Intel® Cor...",,Computers & Electronics>Computers>PCs/Workstat...,HP OMEN 880-156nf 8th gen Intel® Core™ i7 i7-8...,hp omen gen gb gb black tower pc hp omen proce...


PHASE 2 : KeyWord Extraction

In [37]:
from keybert import KeyBERT

In [57]:
import sys
print(sys.executable)


c:\ProgramData\anaconda3\python.exe


In [58]:
# import torch

# print("Torch version:", torch.__version__)
# print("CUDA available:", torch.cuda.is_available())
# print("CUDA version:", torch.version.cuda)
# # print("GPU name:", torch.cuda.get_device_name(0))


In [59]:
# from keybert import KeyBERT
# from sentence_transformers import SentenceTransformer

# # Load SentenceTransformer model with GPU support
# model = SentenceTransformer('all-MiniLM-L6-v2')
# model = model.to('cuda')  # Move model to GPU

# # Pass GPU-enabled model to KeyBERT
# kw_model = KeyBERT(model)

In [68]:
# Load model once
kw_model = KeyBERT(model='all-MiniLM-L6-v2')  # small but powerful

# Example for one row
keywords = kw_model.extract_keywords(df_train['cleaned_text'].iloc[0], top_n=10)
print(keywords)


[('hardware', 0.4942), ('audio', 0.4525), ('vivopc', 0.4499), ('asus', 0.4283), ('streaming', 0.356), ('ram', 0.353), ('multimedia', 0.3468), ('processor', 0.3449), ('hdmi', 0.3436), ('hdtv', 0.3423)]


In [ ]:
# Load model once
kw_model = KeyBERT(model='paraphrase-MiniLM-L3-v2')  # faster, still good quality

# Example for one row
keywords = kw_model.extract_keywords(df_train['cleaned_text'].iloc[0], top_n=10)
print(keywords)


[('gb', 0.4296), ('ram', 0.3824), ('audio', 0.3661), ('bass', 0.36), ('hardware', 0.3524), ('pc', 0.3302), ('processor', 0.3298), ('hdtv', 0.3238), ('amplifier', 0.322), ('sound', 0.3113)]


In [64]:
from keybert import KeyBERT

# Initialize KeyBERT model
kw_model = KeyBERT(model='paraphrase-MiniLM-L3-v2')

# Extract keywords from the first row
sample_text = df_train['cleaned_text'].iloc[0]

keywords = kw_model.extract_keywords(
    sample_text,
    keyphrase_ngram_range=(1, 2),  # Only unigrams & bigrams
    stop_words='english',
    top_n=10
)

# Print the extracted keywords
print([kw[0] for kw in keywords])


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\Amroy\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Amroy\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L3-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

['pc audio', 'powerful audio', 'audio br', 'tune hardware', 'gb hdd', 'gen gb', 'ram hdd', 'memory gb', 'gb', 'capacity gb']


In [ ]:
# from tqdm import tqdm
# tqdm.pandas()

# def extract_keywords(text, top_n=10):
#     return [kw[0] for kw in kw_model.extract_keywords(
#         text,
#         keyphrase_ngram_range=(1, 1),
#         stop_words='english',
#         top_n=top_n
#     )]

# df_train['keywords'] = df_train['cleaned_text'].progress_apply(lambda x: extract_keywords(x, top_n=10))


In [78]:
from keybert import KeyBERT
from tqdm import tqdm

# Initialize KeyBERT model
kw_model = KeyBERT(model='all-MiniLM-L6-v2')  

# Function to extract keywords (only unigrams)
def extract_keywords(text, top_n=10):
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1, 1),  # Only unigrams
        top_n=top_n
    )
    return [kw[0] for kw in keywords]

# Apply to each dataset
tqdm.pandas()
df_train["keywords"] = df_train["cleaned_text"].progress_apply(lambda x: extract_keywords(x, top_n=10))
df_validate["keywords"] = df_validate["cleaned_text"].progress_apply(lambda x: extract_keywords(x, top_n=10))
df_test["keywords"] = df_test["cleaned_text"].progress_apply(lambda x: extract_keywords(x, top_n=10))

100%|██████████| 90598/90598 [2:44:04<00:00,  9.20it/s]  


In [ ]:
# df_train.to_json("icecat_preprocessed_train_keybert.json")
# df_validate.to_json("icecat_preprocessed_validate_keybert.json")
# df_test.to_json("icecat_preprocessed_test_keybert.json")

In [81]:
df_train.head()

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text,cleaned_text,keywords
0,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,<b>Smart Multimedia Performance</b><br>\nVivoP...,ASUS K31CD-IT049T. Processor frequency: 3.4 GH...,"ASUS K31CD-IT049T, 3.4 GHz, 6th gen Intel® Cor...","Intel Core i7-6700 (8M Cache, 3.4GHz), 16GB RA...",Computers & Electronics>Computers>PCs/Workstat...,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,asus pc gen gb gb hdd black tower b smart mult...,"[hardware, audio, vivopc, asus, streaming, ram..."
1,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,Get the performance you demand at a price that...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,1m ST/SC Plenum-Rated 9/125 Duplex Single-Mode...,Computers & Electronics>Computer Cables>Fibre ...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,duplex fiber patch cable blue fiber optic cabl...,"[ethernet, fiber, cable, gigabit, duplex, tran..."
2,HP FA889AA Battery,Keeping an extra source of power nearby means ...,"HP FA889AA. Product type: Battery, Product col...","HP FA889AA, Battery, White, Lithium-Ion (Li-Io...","1100 mAh, Lithium Ion, Standard Battery",Computers & Electronics>Computers>Handheld Mob...,HP FA889AA Battery Keeping an extra source of ...,hp battery keeping extra source power nearby m...,"[hp, battery, power, ipaq, office, standard, l..."
3,Lenovo ThinkStation C30 Intel® Xeon® E5 Family...,The C30 builds on its award-winning design as ...,Lenovo ThinkStation C30. Processor frequency: ...,"Lenovo ThinkStation C30, 2 GHz, Intel® Xeon® E...","Intel Xeon E5-2620 (15M Cache, 2.00 GHz, 7.20 ...",Computers & Electronics>Computers>PCs/Workstat...,Lenovo ThinkStation C30 Intel® Xeon® E5 Family...,lenovo thinkstation family gb gb hdd black tow...,"[lenovo, hardware, workstation, intel, process..."
4,Lenovo 00D8199 software license/upgrade,SUSE Linux Enterprise Server (SLES)<br>\n<br>\...,Lenovo 00D8199. License term in years: 3 year(s),"Lenovo 00D8199, 3 year(s)",SLES for SAP Applications 2 Socket 24x7 SUSE S...,Computers & Electronics>Software>Software Lice...,Lenovo 00D8199 software license/upgrade SUSE L...,lenovo software suse linux enterprise server s...,"[sap, linux, lenovo, unix, server, enterprise,..."


In [82]:
df_train[['cleaned_text',"keywords"]]

,cleaned_text,keywords
0,asus pc gen gb gb hdd black tower b smart mult...,"[hardware, audio, vivopc, asus, streaming, ram..."
1,duplex fiber patch cable blue fiber optic cabl...,"[ethernet, fiber, cable, gigabit, duplex, tran..."
2,hp battery keeping extra source power nearby m...,"[hp, battery, power, ipaq, office, standard, l..."
3,lenovo thinkstation family gb gb hdd black tow...,"[lenovo, hardware, workstation, intel, process..."
4,lenovo software suse linux enterprise server s...,"[sap, linux, lenovo, unix, server, enterprise,..."
...,...,...
289860,duracell aaa battery alkaline duracell aaa bat...,"[alkaline, aaa, battery, cylindrical, duracell..."
289861,velocity camcorder cable x rca black camcorder...,"[camcorder, cable, vcr, rca, television, veloc..."
289862,fujitsu esprimo gen gb gb ssd black red midi t...,"[fujitsu, ssd, processor, intel, desktop, pc, ..."
289863,gigabyte aire mouse usb optical dpi b mini siz...,"[usb, gigabyte, cord, mouse, notebook, cable, ..."


In [86]:
df_train.to_csv("icecat_keywords_train.csv", index=False)
df_validate.to_csv("icecat_keywords_validate.csv", index=False)
df_test.to_csv("icecat_keywords_test.csv", index=False)


In [ ]:
# df_train.to_json("icecat_keywords_train_orient.json", orient="records", lines=True)
# df_validate.to_json("icecat_keywords_train_orient.json", orient="records", lines=True)
# df_test.to_json("icecat_keywords_train_orient.json", orient="records", lines=True)
